In [1]:
def loadCoreArticle(string):    
    f = open('/home/lr/Downloads/conference_data/' + string + '.txt','r')
    sample = f.readlines()

    dict_finallis_keyword = eval(sample[0])
    f.close()


    return dict_finallis_keyword
"""
tochi_core = loadCoreArticle('tochi_core')
avi_core = loadCoreArticle('avi_core')
icmi_core = loadCoreArticle('icmi_core')
gi_core = loadCoreArticle('gi_core')
chi_core = loadCoreArticle('chi_core')
idc_core = loadCoreArticle('idc_core')
lak_review = loadCoreArticle('lak_review')
"""

cof = ['assets','avi','chi','cscw','dis','gi','group','icmi','idc','its','iui','lak','ls','mobilechi','tochi','ubicomp','uist']

In [2]:
import copy
title_list = []
reference_list = {}
for s in cof:
    name = s + "_core"
    
    data = loadCoreArticle(name)
    #print (len(data))
    title_list.extend(list(data.keys()))
    for i in data:
        reference_list[i] = copy.deepcopy(data[i])

In [3]:
title_list[8] = ": designing a new class of computational toys"
reference_list[title_list[8]] = reference_list['curlybot: designing a new class of computational toys']
title_list[16] = ": Haptic Feedback to Enhance Early Reading"
reference_list[title_list[16]] = reference_list['FeelSleeve: Haptic Feedback to Enhance Early Reading']
title_list[32] = ": The Effects of Curriculum-Aligned Making on Children's Self-Identity"
reference_list[title_list[32]] = reference_list["I Make, Therefor I Am: The Effects of Curriculum-Aligned Making on Children's Self-Identity"]

In [4]:
#获取数据
from elasticsearch import helpers
import elasticsearch


total_text = {}
total_bibliometrics = {}
total_index_term = {}
total_reference = {}
total_author = {}
article_author = {}
total_article = {}
total_id = {}
total_conf = {}

ES_SERVERS = [{
    # 'host': 'localhost',
    'host': '127.0.0.1',
    'port': 9200
}]

es_client = elasticsearch.Elasticsearch(
    hosts=ES_SERVERS
)

es_result = helpers.scan(
    client=es_client,
    query={
    "query": {
        "match_all": {}
    }
},
    scroll='5m',
    index='conference_total',
    #doc_type='conference_total',
    # timeout='1m'
)
# print(len(es_result))

#h_heads=[]
cont=0
for i in es_result:
    data_year = i['_source']
    h_head = i['_source']['h_head']
    id_num = i['_id']
    year = data_year['h_year']
    if year not in total_article.keys():
        total_article[year] = {'title':[], 'abstract':[]}
    #if 'group' not in data_year.keys():
        #print (h_head)
       

    

           
    author = data_year['author']
    #reference = data_year['reference']
    index_term = data_year['index_term']
    bibliometrics = data_year['citition']
    title = data_year['title']
    abstract = data_year['abstract']
    if title not in title_list:
        continue
        
    
    cont += 1 
    total_id.setdefault(year,[]).append(id_num)
    total_bibliometrics.setdefault(year,[]).append(bibliometrics)
    total_reference.setdefault(year,[]).append(reference_list[title])
    total_index_term.setdefault(year,[]).append(index_term)
    total_article[year]['title'].append(title)
    total_article[year]['abstract'].append(abstract)
    article_author.setdefault(year,[]).append(author)
    total_conf.setdefault(year,[]).append(h_head)

    
author_data = article_author
print (cont)

164


# 对比共同作者

In [5]:
import copy
#初始化列表
num_same_author = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_author.setdefault(s,[]).append(copy.deepcopy(single))
        
#print (num_same_author)

for s in author_data:
    count = 0
    for i in author_data[s]:
        for j in i:
            name1 = j['name']       
            
            for m in author_data:
                temp = -1
                for n in author_data[m]:
                    temp += 1
                    for o in n:
                        name2 = o['name']
                        
                        if name1 == name2:
                            num_same_author[s][count][m][temp] += 1
                            
                           
        count += 1

# 对比共同标签

In [6]:
import copy
#初始化列表
num_same_index = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_index.setdefault(s,[]).append(copy.deepcopy(single))
        
for s in total_index_term:
    count = 0
    for i in total_index_term[s]:
        #print (i)
        for j in i:    
            
            for m in total_index_term:
                temp = -1
                for n in total_index_term[m]:
                    temp += 1
                    for o in n:                        
                        if j == o:
                            num_same_index[s][count][m][temp] += 1
                            
                           
        count += 1


# 对比共同关键词

In [7]:
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
import numpy as np
import re
import math

#设置stopwords，缩略词拆解

stopwords = set (stopwords.words('english')+list(punctuation)+[').'])


#分段成句和分句成词
def splitSentence(paragraph):
    sent = sent_tokenize(paragraph)
    return sent


def wordtokenizer(sentence):
    word = []
    words = WordPunctTokenizer().tokenize(sentence)
    word = word + words
    return word

#文章预处理
def standardization(word_sent):
#转换为小写字母    
    text = []
    for s in word_sent:
        text.append(s.lower())
    
    return text

#词形还原
def lemmatizer(word):
    wnl = WordNetLemmatizer()
    text = []
#对单词类型标注
    text_tag = np.array(nltk.pos_tag(word))
    #print (text_tag)
    
    for s in text_tag:
        if s[1].startswith('N'):
            if s[1] == 'NNS' or s[1] == 'NN':
                text.append(wnl.lemmatize(s[0],'n'))
            else:
                text.append(s[0])
        elif s[1].startswith('V'):
            text.append(wnl.lemmatize(s[0], 'v'))
        elif s[1].startswith('J'):
            text.append(wnl.lemmatize(s[0], 'a'))
        elif s[1].startswith('R'):
            text.append(wnl.lemmatize(s[0], 'r'))
        else:
            text.append(s[0])
            
    #print (text)
    new_text = [word for word in text if word not in stopwords and 3<len(word)]
    
    return new_text

def filtrated(word):
    wnl = WordNetLemmatizer()
    text = []
#对单词类型标注
    text_tag = np.array(nltk.pos_tag(word))
    #print (text_tag)
    
    for s in text_tag:
        if s[1].startswith('N'):            
            text.append(s[0])
        
    
    return text

#统计词频
def wordFreq(word):
    word_Freq = {}
    for s in word:
        if not s in word_Freq:
            word_Freq[s] = 1
        else:
            word_Freq[s] += 1
    
    num = 0
    for key,value in word_Freq.items():
        num = num + value
    
    return word_Freq, num


In [8]:
#提取关键词
from sklearn.feature_extraction.text import CountVectorizer
education_keywords = ['education','learn','teach','educate','exam','student','curriculum','course']

keyword_sum = {}
for s in total_article:
    for i in range(len(total_article[s]['title'])):
        if total_article[s]['abstract'][i] != []:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i] + " " + total_article[s]['abstract'][i]
        else:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i]
            
        sent = splitSentence(text) 
        
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word
        #print (word_list)
        word_standard = standardization(word_list)
        #print (word_standard)
        word_ori = lemmatizer(word_standard)
        
        single = filtrated(word_ori)
        word_Freq, num = wordFreq(single)
        
        temp = []
        for j in word_Freq:
            if word_Freq[j] > 1 and j not in education_keywords:
                temp.append(j)
        keyword_sum.setdefault(s,[]).append(temp)
        #print (temp)

In [9]:
import copy
#初始化列表
num_same_keyword = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_keyword.setdefault(s,[]).append(copy.deepcopy(single))
        
for s in keyword_sum:
    count = 0
    for i in keyword_sum[s]:
        for j in i:    
            
            for m in keyword_sum:
                temp = -1
                for n in keyword_sum[m]:
                    temp += 1
                    for o in n:                        
                        if j == o:
                            num_same_keyword[s][count][m][temp] += 1
                            break
                            
                           
        count += 1


# 对比共同参考文献

In [10]:
import copy
#初始化列表
num_same_ref = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_ref.setdefault(s,[]).append(copy.deepcopy(single))
        

for s in total_reference:
    count = 0
    for i in total_reference[s]:
        #print (i)
        for j in i['reference']:    
            
            for m in total_reference:
                temp = -1
                for n in total_reference[m]:
                    temp += 1
                    for o in n['reference']:                        
                        if j == o:
                            num_same_ref[s][count][m][temp] += 1
                            break
                            
                           
        count += 1
        
        

# 计算相似得分

In [196]:
import copy
#初始化列表
sim_score = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        sim_score.setdefault(s,[]).append(copy.deepcopy(single))
a = 5
b = 5
c = 0
d = 0

for s in total_article:
    for i in range(len(total_article[s]['title'])):
        
        for year in num_same_author[s][i]:
            for j in range(len(num_same_author[s][i][year])):
                same_ref = num_same_ref[s][i][year][j]
                same_author = num_same_author[s][i][year][j]
                same_index = num_same_index[s][i][year][j]
                same_keyword = num_same_keyword[s][i][year][j]
                
                sim_score[s][i][year][j] = a*same_ref + d*same_keyword + c*same_index + b*same_author  
                
                
        


# 生成力导向数据

In [197]:
import pandas as pd
import numpy as np
df = pd.DataFrame(columns = np.arange(164))
article_position = {}
num = 0
for s in sim_score:
    count = 0
    for i in sim_score[s]:
        title = total_article[s]['title'][count]
        article_position[title] = num
        temp = []
        for j in i:
            temp.extend(i[j])
        #print (temp)
        df.loc[num] = temp
        
        count += 1
        num += 1
#print (df)

In [198]:
t = []
for s in df:
    #if s > 100:
        #break
    for i in df:
        #if i > 100:
            #break
        if s != i and df[s][i] > 14:
            print (str(s) + ' -- ' + str(i) + " {weight: " + str(df[s][i]/10) + "}")
            t.append(s)
            t.append(i)
s2 = list(set(t))

1 -- 34 {weight: 3.0}
5 -- 15 {weight: 1.5}
5 -- 36 {weight: 1.5}
5 -- 54 {weight: 1.5}
6 -- 137 {weight: 1.5}
6 -- 142 {weight: 1.5}
11 -- 24 {weight: 2.0}
11 -- 34 {weight: 1.5}
11 -- 52 {weight: 2.5}
11 -- 92 {weight: 2.0}
11 -- 94 {weight: 1.5}
11 -- 137 {weight: 1.5}
11 -- 143 {weight: 3.5}
14 -- 51 {weight: 2.5}
15 -- 5 {weight: 1.5}
15 -- 54 {weight: 1.5}
17 -- 49 {weight: 1.5}
17 -- 59 {weight: 1.5}
17 -- 146 {weight: 1.5}
21 -- 74 {weight: 1.5}
23 -- 96 {weight: 2.0}
24 -- 11 {weight: 2.0}
24 -- 143 {weight: 1.5}
31 -- 146 {weight: 1.5}
31 -- 159 {weight: 1.5}
34 -- 1 {weight: 3.0}
34 -- 11 {weight: 1.5}
34 -- 40 {weight: 1.5}
34 -- 52 {weight: 1.5}
34 -- 64 {weight: 1.5}
36 -- 5 {weight: 1.5}
36 -- 54 {weight: 1.5}
36 -- 74 {weight: 1.5}
36 -- 87 {weight: 2.0}
37 -- 60 {weight: 1.5}
40 -- 34 {weight: 1.5}
45 -- 55 {weight: 1.5}
45 -- 147 {weight: 1.5}
47 -- 61 {weight: 2.0}
47 -- 72 {weight: 2.5}
47 -- 147 {weight: 2.5}
49 -- 17 {weight: 1.5}
49 -- 50 {weight: 3.5}
49 -- 59 {

In [199]:
"""for s in range(164):
    if s not in s2:
        print (s)"""
print (len(s2))

81


In [20]:
t = []
node_temp = {}
edges_temp = []
count = 0
for s in df:
    #if s > 100:
        #break
    for i in df:
        #if i > 100:
            #break
        if s != i and df[s][i] > 10:
            if s not in node_temp.keys():
                node_temp[s] = count
                count += 1
            if i not in node_temp.keys():
                node_temp[i] = count
                count += 1
            edges_temp.append({"source": s, "target": i, "weight": df[s][i]/2})
                
s2 = list(set(t))

In [23]:
node = []
edge = []
for s in node_temp:
    node.append({"name:": s})
    
print (node)
for s in edges_temp:
    source = s["source"]
    target = s['target']
    edge.append({"source": node_temp[source], "target": node_temp[target], "weight": s['weight']})
    print (edge)

[{'name:': 11}, {'name:': 143}, {'name:': 64}, {'name:': 76}, {'name:': 65}, {'name:': 84}, {'name:': 68}, {'name:': 77}, {'name:': 85}, {'name:': 113}, {'name:': 108}, {'name:': 116}, {'name:': 138}, {'name:': 139}]
[{'source': 0, 'target': 1, 'weight': 7.0}, {'source': 2, 'target': 3, 'weight': 11.0}, {'source': 4, 'target': 5, 'weight': 7.0}, {'source': 6, 'target': 7, 'weight': 12.0}, {'source': 3, 'target': 2, 'weight': 11.0}, {'source': 7, 'target': 6, 'weight': 12.0}, {'source': 5, 'target': 4, 'weight': 7.0}, {'source': 8, 'target': 9, 'weight': 6.0}, {'source': 10, 'target': 11, 'weight': 12.0}, {'source': 9, 'target': 8, 'weight': 6.0}, {'source': 11, 'target': 10, 'weight': 12.0}, {'source': 12, 'target': 13, 'weight': 6.0}, {'source': 13, 'target': 12, 'weight': 6.0}, {'source': 1, 'target': 0, 'weight': 7.0}]


In [ ]:
print (df[95])

# 修正引用数

In [154]:
now = 2019
index = 4
high_cited = {}
missing = {}
new_cite_list = []
for s in total_bibliometrics:
    #print (s)
    #print (total_bibliometrics[s])
    count = 0
    if s == "2014/2015":
        year = 2014.0
    else:
        year = float(s)
    for i in total_bibliometrics[s]:
        #print (i)

        new_cite = index / (now - year + 1) * (i + 1)
        print (new_cite)
        new_cite_list.append(new_cite)
print (len(new_cite_list))

18.666666666666664
12.0
29.333333333333332
13.333333333333332
12.0
12.0
21.333333333333332
12.0
13.333333333333332
13.333333333333332
12.0
12.0
13.333333333333332
12.0
12.0
15.0
1.0
20.0
13.0
13.0
18.0
12.0
13.0
13.0
25.0
19.0
14.0
18.0
12.0
26.0
12.0
12.0
13.0
16.0
22.0
14.0
24.0
36.0
12.8
26.400000000000002
15.200000000000001
17.6
11.200000000000001
16.8
11.200000000000001
13.600000000000001
17.6
14.4
14.4
12.8
11.200000000000001
14.4
34.4
12.0
21.6
22.400000000000002
13.600000000000001
16.0
12.8
22.400000000000002
24.0
18.400000000000002
19.200000000000003
28.8
11.200000000000001
30.400000000000002
12.666666666666666
30.0
33.33333333333333
66.0
12.666666666666666
20.666666666666664
16.0
16.0
14.666666666666666
14.0
11.333333333333332
29.333333333333332
12.0
16.666666666666664
16.0
25.333333333333332
25.333333333333332
13.333333333333332
88.66666666666666
14.285714285714285
10.857142857142856
69.14285714285714
31.428571428571427
24.57142857142857
16.0
29.142857142857142
14.8571428571

In [161]:
color = ['#97CBFF','#2894FF','#005AB5']

In [165]:
for s in range(len(new_cite_list)):
    if new_cite_list[s] > 40:
        print (str(s) + "{color:" + color[2] + "}")
        continue
    if new_cite_list[s] > 30:
        print (str(s) + "{color:" + color[1] + "}")
        continue
    if new_cite_list[s] > 20:
        print (str(s) + "{color:" + color[0] + "}")
        continue
    

2{color:#97CBFF}
6{color:#97CBFF}
24{color:#97CBFF}
29{color:#97CBFF}
34{color:#97CBFF}
36{color:#97CBFF}
37{color:#2894FF}
39{color:#97CBFF}
52{color:#2894FF}
54{color:#97CBFF}
55{color:#97CBFF}
59{color:#97CBFF}
60{color:#97CBFF}
63{color:#97CBFF}
65{color:#2894FF}
67{color:#97CBFF}
68{color:#2894FF}
69{color:#005AB5}
71{color:#97CBFF}
77{color:#97CBFF}
81{color:#97CBFF}
82{color:#97CBFF}
84{color:#005AB5}
87{color:#005AB5}
88{color:#2894FF}
89{color:#97CBFF}
91{color:#97CBFF}
96{color:#005AB5}
97{color:#97CBFF}
102{color:#97CBFF}
103{color:#97CBFF}
104{color:#2894FF}
105{color:#97CBFF}
111{color:#97CBFF}
131{color:#97CBFF}
132{color:#97CBFF}
133{color:#97CBFF}
137{color:#97CBFF}
145{color:#97CBFF}
149{color:#97CBFF}
151{color:#2894FF}
154{color:#97CBFF}
156{color:#005AB5}
158{color:#97CBFF}
161{color:#2894FF}


# 导出文章信息

In [126]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame(columns = ['index','title','abstract','author','citation','conference','year','id'])

In [178]:

for s in total_article:
    for i in range(len(total_article[s]['title'])):
        temp = []
        title = total_article[s]['title'][i]
        abstract = total_article[s]['abstract'][i]
        author = author_data[s][i]
        index = total_id[s][i]
        conf = total_conf[s][i]
        citation = total_bibliometrics[s][i]
        num = article_position[title]
        
        temp.append(num)
        temp.append(title)
        temp.append(abstract)
        temp.append(author)
        temp.append(citation)
        temp.append(conf)
        temp.append(s)
        temp.append(index)
        
        
        df1.loc[num] = temp
        
        num += 1
        
print (df1)

    index                                              title  \
142   142  Blocks4All: Overcoming Accessibility Barriers ...   
143   143  Bots & (Main)Frames: Exploring the Impact of T...   
144   144  Science Everywhere: Designing Public, Tangible...   
145   145  “Bursting the Assistance Bubble”: Designing In...   
146   146  ConceptScape: Collaborative Concept Mapping fo...   
147   147  Collaborative Live Media Curation: Shared Cont...   
148   148  Co-designing online privacy-related games and ...   
149   149  EmotiW 2018: Audio-Video, Student Engagement a...   
0       0  Enabling Real-Time Adaptivity in MOOCs with a ...   
1       1  Learning to Code in Localized Programming Lang...   
2       2  Writing Reusable Code Feedback at Scale with M...   
3       3  Intelligent tutors as teachers' aides: explori...   
4       4  Trends and issues in student-facing learning a...   
5       5  Follow the successful crowd: raising MOOC comp...   
6       6  Enabling Collaboration in Lea

In [179]:
import os
path = "/home/lr/Downloads/conference_data/education_high_cited/"
if not os.path.exists(path):
     os.mkdir(path)
#date_rand =time.strftime("%Y-%m-%d",time.localtime())
file_name = path+'artical_info.xlsx'
writer = pd.ExcelWriter(file_name)

df1.to_excel(writer, "article_info")
writer.save()

# 聚类分组+词云

In [166]:
onlineLearning = [62,90,61,96,23,83,63,82,72,47,147,45,49,50,54,15,5,69,68,77,59,146,31,159,134]
tabletopInterface = [117,98,108,116,129,139,138]
STEMedu = [1,34,40,52,94,11,92,24,143,137,156,64,76,122,6,142,151,158,140,125]
stuLife = [84,65]
onlineEngag = [87,36,74]

In [ ]:
def generateWC(group):
    word_sum = []
    for s in group:
        position = findPosition(s)
        year = position[0]
        index = position[1]

        title = total_article[year]['title'][index]
        abstract = total_article[year]['title'][index]
        text = title + ' ' + title + ' ' + abstract

        sent = splitSentence(title) 
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word

        word_standard = standardization(word_list)
        word_ori = lemmatizer(word_standard)

        word_sum.extend(word_ori)

    #print (word_sum)
    prep_list = '/'.join(word_sum)
    #print (prep_list)
    
    from wordcloud import WordCloud
    wc = WordCloud(width = 600, height = 300).generate(prep_list)
    image = wc.to_image()
    image.show()

In [183]:
def wordList(group):
    word_sum = []
    for s in group:
        info = df1.loc[s]
        title = info['title']
        abstract = info['abstract']

        text = title + ' ' + title + ' ' + abstract

        sent = splitSentence(title) 
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word

        word_standard = standardization(word_list)
        word_ori = lemmatizer(word_standard)

        word_sum.extend(word_ori)

    prep_list = '/'.join(word_sum)
    return prep_list

ol_list = wordList(onlineLearning)
ti_list = wordList(tabletopInterface)
stem_list = wordList(STEMedu)

In [190]:
def generateWC(prep_list,string):
    from wordcloud import WordCloud
    wc = WordCloud(width = 800, height = 600).generate(prep_list)
    image = wc.to_image()
    wc.to_file(string)
    image.show()
    
generateWC(ol_list,'OnlingLearning.png')
generateWC(ti_list,'tabletopInterface.png')
generateWC(stem_list,'STEM.png')

In [193]:
for s in range(len(new_cite_list)):
    if new_cite_list[s] < 30 and new_cite_list[s] > 20:
        print (s)
        continue
    """if new_cite_list[s] > 30:
        print (str(s) + "{color:" + color[1] + "}")
        continue
    if new_cite_list[s] > 20:
        print (str(s) + "{color:" + color[0] + "}")
        continue"""

2
6
24
29
34
36
39
54
55
59
60
63
71
77
81
82
89
91
97
102
103
105
111
131
132
133
137
145
149
154
158
